In [31]:

import os
import pandas as pd
from readability import Readability
import nltk
from bert_score import score
from pprint import pprint
from botocore.exceptions import ClientError
import openai
from difflib import SequenceMatcher
import re
import mammoth
import openai
from dotenv import load_dotenv

In [32]:
# Load environment variables from .env file
load_dotenv()

openai.api_key = os.environ.get('OPENAI_API_KEY')
openai.organization = 'org-WD3x2XVqrr4UO8u1zjuYSyXZ' 

In [33]:
root_file = 'C:\\Users\\dedbl\\Documents\\NHSPolicy\\'
drug  = 'phenytoin'#'monofer'#'amikacin'

In [38]:
# given this document, which of these questions have not been answered
extra_resource = root_file + 'extra_resources\\'
    
with open(extra_resource+ drug+'.txt', "r") as docx_file:
    info_prompt = docx_file.read()

generate_questions = """You are a nurse administering IV medication,
what questions would you need to be ask a technical expert, to generate the following text? 
Give the output directly as a list of numbered questions - TEXT: """ + info_prompt

questions = openai.ChatCompletion.create(
        model = "gpt-4",
        messages=[{"role": "user", "content": generate_questions}],
        max_tokens = 1000,
        temperature = 0
        )
questions = questions["choices"][0]["message"]["content"]

In [ ]:

print(questions)

1. What is the pharmaceutical form of the medication?
2. What is the appearance of the medication?
3. What is the pH and osmolarity of the reconstituted solution?
4. What are the therapeutic indications of the medication?
5. What is the recommended dosage and method of administration for adults and children?
6. Are there any dosage adjustments for patients with hepatic or renal impairment?
7. What are the contraindications of the medication?
8. What are the special warnings and precautions for use?
9. What are the potential interactions with other medicinal products?
10. What are the effects of the medication on fertility, pregnancy, and lactation?
11. What are the potential effects on the ability to drive and use machines?
12. What are the undesirable effects of the medication?
13. What should be done in case of an overdose?
14. What are the pharmacological properties of the medication?
15. What are the preclinical safety data?
16. What are the pharmaceutical particulars of the medica

In [39]:
# given this document, which of these questions have not been answered

# load up drug guideline
guideline = root_file + drug + "_gpt_formatted.md"
with open(guideline, "r") as file:
    current_guideline = file.read()



examine_text = '''You are a nurse following a guideline for administering IV medication. You need to find questions from the list provided 
which are NOT already answered by the guideline you are given. Find the unanswered questions. 
As well as the question, output section heading from the guideline most related to the question.
The output format will be the heading: followed by the questions.
Only output a maximum of ten questions in total across all sections.
These are the questions ''' + questions + '. This is the guideline text: ' + current_guideline

technical_suggestions = openai.ChatCompletion.create(
        model = "gpt-4",
        messages=[{"role": "user", "content": examine_text}],
        max_tokens = 2500,
        temperature = 0
        )

technical_suggestions = technical_suggestions["choices"][0]["message"]["content"]
print(technical_suggestions)


Preparation: What precautions should be taken when preparing solutions for parenteral administration?

Administration: What is the recommended rate of administration for intravenous phenytoin?

Adverse effects and suggested monitoring: What are the potential undesirable effects of phenytoin?

Other comments: What are the potential effects of phenytoin on fertility, pregnancy, and lactation?

Other comments: What are the potential effects of phenytoin on the ability to drive and use machines?

Other comments: What is the recommended course of action in case of an overdose of phenytoin?

Other comments: What are the pharmacodynamic properties of phenytoin?

Other comments: What are the preclinical safety data for phenytoin?

Other comments: What are the pharmaceutical particulars of phenytoin?

Compatibility: What are the potential incompatibilities of phenytoin with other drugs?
